In [2]:
from __future__ import annotations
import os
from datetime import date, timedelta
from typing import List, Dict, Any
import httpx
import requests
import pandas as pd
import json
import unicodedata

from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import math, requests
from datetime import date, timedelta

RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")
HOST = "apidojo-booking-v1.p.rapidapi.com"
BASE = f"https://{HOST}"
HEADERS = {"X-RapidAPI-Key": RAPIDAPI_KEY, "X-RapidAPI-Host": HOST}

def bbox_from_center(lat, lon, km=8):
    # approx degrés par km
    dlat = km / 110.574
    dlon = km / (111.320 * math.cos(math.radians(lat)))
    return (lat + dlat, lon + dlon, lat - dlat, lon - dlon)  # NE_lat, NE_lon, SW_lat, SW_lon

def list_hotels_by_map(center_lat, center_lon, km=8,
                       checkin=None, checkout=None,
                       adults=2, children_qty=0,
                       currency="EUR", language="fr-fr",
                       page_number=0):
    if checkin is None:
        checkin = (date.today() + timedelta(days=14)).isoformat()
    if checkout is None:
        checkout = (date.today() + timedelta(days=17)).isoformat()

    ne_lat, ne_lon, sw_lat, sw_lon = bbox_from_center(center_lat, center_lon, km)
    url = f"{BASE}/properties/v2/list-by-map"
    params = {
        "ne_latitude": f"{ne_lat:.6f}",
        "ne_longitude": f"{ne_lon:.6f}",
        "sw_latitude": f"{sw_lat:.6f}",
        "sw_longitude": f"{sw_lon:.6f}",
        "checkin_date": checkin,
        "checkout_date": checkout,
        "adults_number": adults,
        "children_qty": children_qty,
        "room_number": 1,
        "order_by": "popularity",
        "languagecode": language,
        "price_filter_currencycode": currency,
        "page_number": page_number
    }
    r = requests.get(url, headers=HEADERS, params=params, timeout=60)
    r.raise_for_status()
    data = r.json() or {}
    return data.get("result") or data.get("results") or []

# --- Démo : Paris centre ---
PARIS_LAT, PARIS_LON = 48.8566, 2.3522
hotels = list_hotels_by_map(PARIS_LAT, PARIS_LON, km=8)

for h in hotels[:15]:
    name = h.get("hotel_name") or h.get("name")
    price = h.get("min_total_price") or (h.get("price_breakdown") or {}).get("all_inclusive_price")
    print(name, "-", price)


RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")  # <-- mets ta clé ici

HOST = "apidojo-booking-v1.p.rapidapi.com"
BASE = f"https://{HOST}"
HEADERS = {"X-RapidAPI-Key": RAPIDAPI_KEY, "X-RapidAPI-Host": HOST}

def bbox_from_center(lat, lon, km=8):
    # approx degrés par km
    dlat = km / 110.574
    dlon = km / (111.320 * math.cos(math.radians(lat)))
    return (lat + dlat, lon + dlon, lat - dlat, lon - dlon)  # NE_lat, NE_lon, SW_lat, SW_lon

def list_hotels_by_map(center_lat, center_lon, km=8,
                       checkin=None, checkout=None,
                       adults=2, children_qty=0,
                       currency="EUR", language="fr-fr",
                       page_number=0):
    if checkin is None:
        checkin = (date.today() + timedelta(days=14)).isoformat()
    if checkout is None:
        checkout = (date.today() + timedelta(days=17)).isoformat()

    ne_lat, ne_lon, sw_lat, sw_lon = bbox_from_center(center_lat, center_lon, km)
    url = f"{BASE}/properties/v2/list-by-map"
    params = {
        "ne_latitude": f"{ne_lat:.6f}",
        "ne_longitude": f"{ne_lon:.6f}",
        "sw_latitude": f"{sw_lat:.6f}",
        "sw_longitude": f"{sw_lon:.6f}",
        "checkin_date": checkin,
        "checkout_date": checkout,
        "adults_number": adults,
        "children_qty": children_qty,
        "room_number": 1,
        "order_by": "popularity",
        "languagecode": language,
        "price_filter_currencycode": currency,
        "page_number": page_number
    }
    r = requests.get(url, headers=HEADERS, params=params, timeout=60)
    r.raise_for_status()
    data = r.json() or {}
    return data.get("result") or data.get("results") or []

# --- Démo : Paris centre ---
PARIS_LAT, PARIS_LON = 48.8566, 2.3522
hotels = list_hotels_by_map(PARIS_LAT, PARIS_LON, km=8)

for h in hotels[:15]:
    name = h.get("hotel_name") or h.get("name")
    price = h.get("min_total_price") or (h.get("price_breakdown") or {}).get("all_inclusive_price")
    print(name, "-", price)


In [54]:
import os, math, requests, pandas as pd

# ==== CONFIG ====
ARRIVAL_DATE = "2025-10-15"
DEPARTURE_DATE = "2025-10-18"
RADIUS_KM    = 10            
CURRENCY     = "EUR"
LANGUAGE     = "fr-fr"
MAX_PAGES    = 30            
GUEST_QTY    = 2
ROOM_QTY     = 1
# ===============

RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")
if not RAPIDAPI_KEY:
    raise RuntimeError("RAPIDAPI_KEY manquante dans l'environnement.")

BOOKING_HOST = "apidojo-booking-v1.p.rapidapi.com"
BOOKING_BASE = f"https://{BOOKING_HOST}"
BOOKING_HEADERS = {"X-RapidAPI-Key": RAPIDAPI_KEY, "X-RapidAPI-Host": BOOKING_HOST}

def geocode_city(city: str, country: str | None = None):
    """Géocode via Nominatim (OpenStreetMap)."""
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": f"{city}, {country}" if country else city, "format": "json", "limit": 1}
    headers = {"User-Agent": "city-geocoder/1.0 (contact: you@example.com)"}
    r = requests.get(url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    hits = r.json()
    if not hits:
        raise ValueError(f"Ville introuvable: {city!r}")
    return float(hits[0]["lat"]), float(hits[0]["lon"])

def bbox_from_center(lat: float, lon: float, km: float):
    dlat = km / 110.574
    dlon = km / (111.320 * math.cos(math.radians(lat)))
    return f"{lat-dlat:.6f},{lat+dlat:.6f},{lon-dlon:.6f},{lon+dlon:.6f}"  # lat_min,lat_max,lon_min,lon_max

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dp = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dp/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dl/2)**2
    return 2*R*math.asin(math.sqrt(a))

def fetch_hotels_by_city(city: str, country: str | None = None) -> pd.DataFrame:
    lat_c, lon_c = geocode_city(city, country)
    bbox = bbox_from_center(lat_c, lon_c, RADIUS_KM)

    url = f"{BOOKING_BASE}/properties/v2/list-by-map"
    rows = []

    for page in range(MAX_PAGES):
        offset = page * 25
        params = {
            "bbox": bbox,
            "room_qty": str(ROOM_QTY),
            "guest_qty": str(GUEST_QTY),
            "arrival_date": ARRIVAL_DATE,
            "departure_date": DEPARTURE_DATE,
            "order_by": "popularity",
            "languagecode": LANGUAGE,
            "price_filter_currencycode": CURRENCY,
            "offset": str(offset),
        }
        r = requests.get(url, headers=BOOKING_HEADERS, params=params, timeout=60)
        r.raise_for_status()
        results = (r.json() or {}).get("result") or []

        if not results:
            break

        for h in results:
            # skip placeholders vides
            if not h.get("hotel_id") and not h.get("hotel_name"):
                continue

            lat = h.get("latitude") or h.get("hotel_latitude")
            lon = h.get("longitude") or h.get("hotel_longitude")
            dist = None
            if lat is not None and lon is not None:
                try:
                    dist = round(haversine_km(lat_c, lon_c, float(lat), float(lon)), 2)
                except Exception:
                    dist = None

            price = h.get("min_total_price")
            if price is None:
                price = (h.get("price_breakdown") or {}).get("all_inclusive_price")

            rows.append({
                "hotel_id": h.get("hotel_id"),
                "name": h.get("hotel_name") or h.get("name"),
                # "address": h.get("address_trans"),   # <- retiré comme demandé
                "stars": h.get("class") or h.get("hotel_class"),
                "accommodation_type": h.get("accommodation_type_name") or h.get("accommodation_type"),
                "review_score": h.get("review_score"),
                "review_count": h.get("review_nr"),
                "min_total_price": price,
                "currency": CURRENCY,
                "latitude": lat,
                "longitude": lon,
                "distance_km_from_center": dist,
                "city": city,                 # <- la ville passée en entrée
                "site": "booking.com",        # <- constant
            })

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # dédup + tri
    if "hotel_id" in df.columns:
        df = df.drop_duplicates(subset=["hotel_id"], keep="first")
    df = df.sort_values(by=["min_total_price", "distance_km_from_center", "review_score"],
                        ascending=[True, True, False], na_position="last").reset_index(drop=True)
    return df

# --- Exemple ---
# df = fetch_hotels_by_city("Paris", country="France")
# df.head(15)


In [55]:
df_paris = fetch_hotels_by_city("Paris", country="France")
df_paris



,hotel_id,name,stars,accommodation_type,review_score,review_count,min_total_price,currency,latitude,longitude,distance_km_from_center,city,site
0,1756305,ibis budget Vitry sur Seine N7,2.0,Hôtel,7.8,296.0,150.3000,EUR,48.778587,2.375907,9.82,Paris,booking.com
1,11726259,Joli,NaN,Appartement,7.9,67.0,150.6600,EUR,48.897154,2.308002,4.34,Paris,booking.com
2,657164,hotelF1 Paris Saint Denis Université,1.0,Hôtel,5.5,2120.0,157.2800,EUR,48.938092,2.368237,9.49,Paris,booking.com
3,11369421,Chambre Double a proximité de Paris,NaN,B&B / Chambre d'hôtes,8.2,54.0,161.4601,EUR,48.917231,2.365746,7.30,Paris,booking.com
4,11365539,Cœur,NaN,Appartement,7.3,9.0,162.0000,EUR,48.897154,2.308002,4.34,Paris,booking.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,182250,Hôtel des Saints Pères - Esprit de France,4.0,Hôtel,9.0,574.0,1510.4000,EUR,48.853331,2.329771,0.94,Paris,booking.com
348,2169237,Hôtel National Des Arts et Métiers,4.0,Hôtel,8.5,730.0,1548.3600,EUR,48.865761,2.353084,2.54,Paris,booking.com
349,258579,Grand Hôtel Malher,3.0,Hôtel,8.7,960.0,1552.5000,EUR,48.855697,2.361157,3.03,Paris,booking.com
350,26730,Pullman Paris Tour Eiffel,4.0,Hôtel,8.3,6109.0,1629.2800,EUR,48.855604,2.292562,2.04,Paris,booking.com


In [59]:
df_paris[df_paris["name"].str.contains("Aiglon", case=False)]

,hotel_id,name,stars,accommodation_type,review_score,review_count,min_total_price,currency,latitude,longitude,distance_km_from_center,city,site
237,54320,Hôtel Aiglon,4.0,Hôtel,8.7,1837.0,772.7401,EUR,48.839712,2.330173,2.26,Paris,booking.com


In [51]:
df_riyadh = fetch_hotels_by_city("La Mecque", country="Saudi Arabia")
df_riyadh

,hotel_id,name,stars,accommodation_type,review_score,review_count,min_total_price,currency,latitude,longitude,distance_km_from_center,city,site
0,13759228,زاد اليقين,NaN,Hôtel,6.7,24.0,101.4000,EUR,21.440581,39.795162,3.95,La Mecque,booking.com
1,13265215,Al Zaereen Hotel 2,NaN,Hôtel,7.9,925.0,115.4700,EUR,21.410514,39.861993,3.81,La Mecque,booking.com
2,13896148,فندق السنابل الذهبية اثنين للخدمات الفندقية,1.0,Hôtel,6.9,78.0,117.3913,EUR,21.400488,39.877184,5.68,La Mecque,booking.com
3,14717048,فندق الرسالة رويال,1.0,Hôtel,8.0,25.0,121.5000,EUR,21.450486,39.839344,3.54,La Mecque,booking.com
4,14674591,فندق مشرب رواء المنصور Mashrab Riwa Hotel,NaN,Appartement,8.0,1.0,125.7304,EUR,21.414304,39.805970,2.28,La Mecque,booking.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,14708108,فندق درة حسن,1.0,Hôtel,NaN,NaN,4173.9130,EUR,21.401432,39.819202,2.30,La Mecque,booking.com
183,12580777,فيلا خاصة في آي بي إطلالة على الحرم - و عنوانه...,NaN,Villa,9.3,12.0,22004.0700,EUR,21.419937,39.824233,0.29,La Mecque,booking.com
184,10741312,فندق درةطيبة المسجد القطري,1.0,Hôtel,7.9,714.0,25650.0000,EUR,21.419526,39.854372,2.85,La Mecque,booking.com
185,11579858,Bakkah Royal Hotel Makkah by Moro,3.0,Hôtel,7.1,271.0,27000.0000,EUR,21.413400,39.829644,0.88,La Mecque,booking.com


In [52]:
df_riyadh[df_riyadh["name"].str.contains("Omar", case=False)]

,hotel_id,name,stars,accommodation_type,review_score,review_count,min_total_price,currency,latitude,longitude,distance_km_from_center,city,site
163,5148243,Doubletree By Hilton Jabal Omar Makkah,4.0,Hôtel,8.5,8308.0,1746.77,EUR,21.423000,39.818293,0.92,La Mecque,booking.com
168,2258749,Hilton Hotel & Convention Jabal Omar Makkah,5.0,Hôtel,8.4,12589.0,2111.86,EUR,21.423000,39.820047,0.75,La Mecque,booking.com
170,9293559,Address Jabal Omar Makkah,5.0,Hôtel,8.8,9929.0,2273.05,EUR,21.422407,39.819156,0.82,La Mecque,booking.com
175,1396427,Jabal Omar Hyatt Regency Makkah,5.0,Hôtel,8.8,22382.0,2658.00,EUR,21.420344,39.822398,0.47,La Mecque,booking.com
177,571831,Hilton Suites Jabal Omar Makkah,5.0,Hôtel,8.9,6623.0,2750.64,EUR,21.421326,39.821610,0.55,La Mecque,booking.com


In [41]:
import requests

url = "https://apidojo-booking-v1.p.rapidapi.com/properties/detail"

querystring = {"dest_ids":"-3727579","hotel_id":"1720410","departure_date":"2022-10-18","arrival_date":"2022-10-15","rec_guest_qty":"2","rec_room_qty":"1","recommend_for":"3","languagecode":"en-us","currency_code":"USD","units":"imperial"}

headers = {
	"x-rapidapi-key": "fd59fa94d0mshd347bab8295d78fp112088jsnd4f2041c7508",
	"x-rapidapi-host": "apidojo-booking-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'code': '500', 'message': 'mobile.hotelPage: arrival date must be today or after today.'}
